In [1]:
#import will2live
import pandas as pd

### PREP DATA

Before jumping into this data, understand the overall transformations that took place to get this data here.

Working on a summary of McEDA data pipeline 

In [2]:
df=pd.read_csv('df_grouped.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head(5)

,wk_ending,itemdesc,totunits,wavg_price,agc_atpd,total_week_units,item_proportion,rev,adus
0,2013-01-12,Beef Category,338249.0,1.51,1338.590323,1499153.0,0.225627,510755.99,48321.285714
1,2013-01-12,Big Mac,29791.0,3.69,1338.590323,1499153.0,0.019872,109928.79,4255.857143
2,2013-01-12,Cheeseburger,53842.0,0.99,1338.590323,1499153.0,0.035915,53303.58,7691.714286
3,2013-01-12,Chicken Category,204602.0,1.98,1338.590323,1499153.0,0.136478,405111.96,29228.857143
4,2013-01-12,Core QP and QPC,25947.0,3.63,1338.590323,1499153.0,0.017308,94187.61,3706.714286


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   wk_ending         2079 non-null   object 
 1   itemdesc          2079 non-null   object 
 2   totunits          2079 non-null   float64
 3   wavg_price        2079 non-null   float64
 4   agc_atpd          2079 non-null   float64
 5   total_week_units  2079 non-null   float64
 6   item_proportion   2079 non-null   float64
 7   rev               2079 non-null   float64
 8   adus              2079 non-null   float64
dtypes: float64(7), object(2)
memory usage: 146.3+ KB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
totunits,2079.0,1.391360e+05,119499.186236,1.278700e+04,3.784300e+04,7.562900e+04,2.659390e+05,5.169220e+05
wavg_price,2079.0,2.090784e+00,1.214214,8.900000e-01,1.000000e+00,1.510000e+00,3.765000e+00,4.330000e+00
agc_atpd,2079.0,1.323332e+03,131.192692,9.607578e+02,1.248989e+03,1.321036e+03,1.413187e+03,1.614613e+03
total_week_units,2079.0,1.530496e+06,113097.606312,1.213436e+06,1.461542e+06,1.529786e+06,1.597661e+06,1.886512e+06
item_proportion,2079.0,9.090909e-02,0.077623,9.329269e-03,2.517122e-02,4.924469e-02,1.815296e-01,3.245958e-01
rev,2079.0,2.507799e+05,198189.114424,1.138043e+04,7.815060e+04,1.392328e+05,4.332399e+05,8.632597e+05
adus,2079.0,1.987658e+04,17071.312319,1.826714e+03,5.406143e+03,1.080414e+04,3.799129e+04,7.384600e+04


In [5]:
df.dtypes

wk_ending            object
itemdesc             object
totunits            float64
wavg_price          float64
agc_atpd            float64
total_week_units    float64
item_proportion     float64
rev                 float64
adus                float64
dtype: object

In [6]:
print(df['wk_ending'].apply(type).value_counts())
print(df['itemdesc'].apply(type).value_counts())


wk_ending
<class 'str'>    2079
Name: count, dtype: int64
itemdesc
<class 'str'>    2079
Name: count, dtype: int64


In [7]:
df['wk_ending'] = pd.to_datetime(df['wk_ending'])



print(df['wk_ending'].head())
print(df.dtypes)


0   2013-01-12
1   2013-01-12
2   2013-01-12
3   2013-01-12
4   2013-01-12
Name: wk_ending, dtype: datetime64[ns]
wk_ending           datetime64[ns]
itemdesc                    object
totunits                   float64
wavg_price                 float64
agc_atpd                   float64
total_week_units           float64
item_proportion            float64
rev                        float64
adus                       float64
dtype: object


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   wk_ending         2079 non-null   datetime64[ns]
 1   itemdesc          2079 non-null   object        
 2   totunits          2079 non-null   float64       
 3   wavg_price        2079 non-null   float64       
 4   agc_atpd          2079 non-null   float64       
 5   total_week_units  2079 non-null   float64       
 6   item_proportion   2079 non-null   float64       
 7   rev               2079 non-null   float64       
 8   adus              2079 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 146.3+ KB


### DATA PREP PART 1 DONE

### DATA PREP PART 2 - COSTS

For this part, we're gonna oversimplify a bit and just have the cost of each item. 
We could in theory expand this to be any level of granularity
    - Restaurant
    
    - General Area
    
    - Week
    
    - Month 
    
    - Year 
    
    Or any combination

For our default values of costs,  we're going to take the average price for each item across our whole dataset and apply costs of 70% to breakfast items, 60% on other items.

There's a few nuances of simplification here ( we've already aggregated data to be across restaurants, and we've already simplified the item categories to have one for breakfast, and a few for the other categories )

This is more to be a proof of concept, that can be expanded upon and made to be more robust.

In [9]:
df.head(1)

,wk_ending,itemdesc,totunits,wavg_price,agc_atpd,total_week_units,item_proportion,rev,adus
0,2013-01-12,Beef Category,338249.0,1.51,1338.590323,1499153.0,0.225627,510755.99,48321.285714


In [10]:
df.head(3)

,wk_ending,itemdesc,totunits,wavg_price,agc_atpd,total_week_units,item_proportion,rev,adus
0,2013-01-12,Beef Category,338249.0,1.51,1338.590323,1499153.0,0.225627,510755.99,48321.285714
1,2013-01-12,Big Mac,29791.0,3.69,1338.590323,1499153.0,0.019872,109928.79,4255.857143
2,2013-01-12,Cheeseburger,53842.0,0.99,1338.590323,1499153.0,0.035915,53303.58,7691.714286


In [11]:
set(df['itemdesc'])

{'Beef Category',
 'Big Mac',
 'Cheeseburger',
 'Chicken Category',
 'Core QP and QPC',
 'Hamburger',
 'Large Burgers',
 'McChicken',
 'McDouble',
 'Total All Day Breakfast Items',
 'Total Fries'}

In [12]:
# Step 1: Calculate the average weighted price for each unique item
average_prices = df.groupby('itemdesc')['wavg_price'].mean().reset_index()

In [13]:
average_prices

,itemdesc,wavg_price
0,Beef Category,1.857460
1,Big Mac,3.905344
2,Cheeseburger,0.999153
3,Chicken Category,2.093439
4,Core QP and QPC,3.994180
5,Hamburger,0.890000
6,Large Burgers,4.031323
7,McChicken,1.049259
8,McDouble,1.206614
9,Total All Day Breakfast Items,1.508624


Alternatively, we here could allow for insertion of a dataframe, or excel document (or any other data form) to get cost data in.

Another option here would be applying these percentages to whatever the price was for the specific item on that specifc week.

However, in this first proof of concept, we want to make it as simple as possible for the user to interact with in the vizualizations and thus refrain from doing so.

Yet another layer of complexity arises from costs varying depending on how many units are produced and so on, but that is beyond the scope of this tool.

In [14]:
# Step 2: Create a column for cost based on whether the item is a breakfast item
# We'll use a simple function that checks the item description and applies the appropriate cost
def calculate_cost(item, price):
    if item == 'Total All Day Breakfast Items':
        return price * 0.70
    else:
        return price * 0.60

In [15]:
# Apply the function to create the cost column
average_prices['cost'] = average_prices.apply(lambda x: calculate_cost(x['itemdesc'], x['wavg_price']), axis=1)

In [16]:
average_prices

,itemdesc,wavg_price,cost
0,Beef Category,1.857460,1.114476
1,Big Mac,3.905344,2.343206
2,Cheeseburger,0.999153,0.599492
3,Chicken Category,2.093439,1.256063
4,Core QP and QPC,3.994180,2.396508
5,Hamburger,0.890000,0.534000
6,Large Burgers,4.031323,2.418794
7,McChicken,1.049259,0.629556
8,McDouble,1.206614,0.723968
9,Total All Day Breakfast Items,1.508624,1.056037


In [17]:
# Step 3: Merge this 'cost' information back into the original dataframe without adding new rows
# This is akin to a VLOOKUP in Excel, we're adding a 'cost' column to 'df' based on 'itemdesc'
df2 = df.merge(average_prices[['itemdesc', 'cost']], on='itemdesc', how='left')

# The 'df' DataFrame now contains the 'cost' column with the cost of each item

In [18]:
df2.head(4)

,wk_ending,itemdesc,totunits,wavg_price,agc_atpd,total_week_units,item_proportion,rev,adus,cost
0,2013-01-12,Beef Category,338249.0,1.51,1338.590323,1499153.0,0.225627,510755.99,48321.285714,1.114476
1,2013-01-12,Big Mac,29791.0,3.69,1338.590323,1499153.0,0.019872,109928.79,4255.857143,2.343206
2,2013-01-12,Cheeseburger,53842.0,0.99,1338.590323,1499153.0,0.035915,53303.58,7691.714286,0.599492
3,2013-01-12,Chicken Category,204602.0,1.98,1338.590323,1499153.0,0.136478,405111.96,29228.857143,1.256063


#### Great, now we have a dataset with simplified cost values

In [19]:
df2['cost_total']=df2['cost']*df2['totunits']
df2['profit']=df2['rev']-df2['cost_total']
df2.head()

,wk_ending,itemdesc,totunits,wavg_price,agc_atpd,total_week_units,item_proportion,rev,adus,cost,cost_total,profit
0,2013-01-12,Beef Category,338249.0,1.51,1338.590323,1499153.0,0.225627,510755.99,48321.285714,1.114476,376970.456952,133785.533048
1,2013-01-12,Big Mac,29791.0,3.69,1338.590323,1499153.0,0.019872,109928.79,4255.857143,2.343206,69806.460349,40122.329651
2,2013-01-12,Cheeseburger,53842.0,0.99,1338.590323,1499153.0,0.035915,53303.58,7691.714286,0.599492,32277.851683,21025.728317
3,2013-01-12,Chicken Category,204602.0,1.98,1338.590323,1499153.0,0.136478,405111.96,29228.857143,1.256063,256993.102603,148118.857397
4,2013-01-12,Core QP and QPC,25947.0,3.63,1338.590323,1499153.0,0.017308,94187.61,3706.714286,2.396508,62182.191429,32005.418571


##### Code to create more user friendly variables
Not currently utilized

In [20]:
# Rename these to more user friendly variables
import pandas as pd

def toggle_column_names(df):
    # Mapping of technical column names to client-friendly names
    name_mapping = {
        'wk_ending': 'Week Ending',
        'itemdesc': 'Item Description',
        'totunits': 'Total Units',
        'wavg_price': 'Weighted Average Price',
        'agc_atpd': 'Average Transactions Per Day',
        'total_week_units': 'Total Weekly Units',
        'item_proportion': 'Weekly Item Proportion',
        'rev': 'Revenue',
        'adus': 'Average Daily Units Sold',
        'cost': 'Cost Per Unit',
        'cost_total': 'Total Cost',
        'profit': 'Profit'
    }
    
    # Reverse mapping for client-friendly to technical
    reverse_mapping = {v: k for k, v in name_mapping.items()}
    
    # Function to switch names based on current columns
    new_columns = {col: name_mapping.get(col, reverse_mapping.get(col, col)) 
                   for col in df.columns}
    
    df.rename(columns=new_columns, inplace=True)

# Example usage:
# Assume 'df' is your DataFrame
# toggle_column_names(df)
# Now 'df' will have the toggled column names


In [21]:
#toggle_column_names(df2)

In [22]:
df2.head(2)

,wk_ending,itemdesc,totunits,wavg_price,agc_atpd,total_week_units,item_proportion,rev,adus,cost,cost_total,profit
0,2013-01-12,Beef Category,338249.0,1.51,1338.590323,1499153.0,0.225627,510755.99,48321.285714,1.114476,376970.456952,133785.533048
1,2013-01-12,Big Mac,29791.0,3.69,1338.590323,1499153.0,0.019872,109928.79,4255.857143,2.343206,69806.460349,40122.329651


In [23]:
df2.to_csv('DataForViz.csv')